In [29]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Bidirectional, Dropout, GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
import pickle

In [30]:
result_dir = "/kaggle/input/workout-video-extracts-complete/Workout_Extract_Complete"

print("Preparing Dataset")
# Prepare the dataset
X, y = [], []
with open(result_dir+"X.txt", "rb") as filex:
  for inpx in pickle.load(filex):
    X.append(inpx)
with open(result_dir+"y.txt", "rb") as filey:
  for inpy in pickle.load(filey):
    y.append(inpy)


Preparing Dataset


In [31]:
U, v = [], []
length = 160
for m, n in zip(X, y):
    if len(m) > length:
        for i in range(len(m)//length):
            U.append(m[i*length:(i+1)*length])
            v.append(n)
    else:
        U.append(m)
        v.append(n)

In [32]:
# X = U
# y = v

In [33]:
sum, count = 0, 0
for x in X:
    if  0 <= len(x) <= 500:
        sum += len(x)
        count+=1
        
print("Avg: ", sum/count)

Avg:  162.97333333333333


In [34]:
print("splitting Dataset")
# Split the dataset into training, validation, and test sets
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

num_landmarks = 33

print("Padding or truncating dataset")
# Pad or truncate sequences to ensure they have the same length
max_seq_length = max([len(seq) for seq in X_train])
print("Maximum sequence length: ", max_seq_length)

fixed_seq_length = 250

# X_train = [seq[:max_seq_length] + [[0] * num_landmarks] * (max_seq_length - len(seq)) for seq in X_train]
# X_val = [seq[:max_seq_length] + [[0] * num_landmarks] * (max_seq_length - len(seq)) for seq in X_val]
# X_test = [seq[:max_seq_length] + [[0] * num_landmarks] * (max_seq_length - len(seq)) for seq in X_test]



X_train = [seq[:max_seq_length] + [[0] * num_landmarks] * (fixed_seq_length - len(seq)) if len(seq) < fixed_seq_length else seq[:fixed_seq_length] for seq in X_train]
X_val = [seq[:max_seq_length] + [[0] * num_landmarks] * (fixed_seq_length - len(seq)) if len(seq) < fixed_seq_length else seq[:fixed_seq_length] for seq in X_val]
X_test = [seq[:max_seq_length] + [[0] * num_landmarks] * (fixed_seq_length - len(seq)) if len(seq) < fixed_seq_length else seq[:fixed_seq_length] for seq in X_test]


splitting Dataset
Padding or truncating dataset
Maximum sequence length:  6838


In [35]:
# from keras.preprocessing.sequence import pad_sequences

# # Define the maximum sequence length
# fixed_seq_length = 150  # Or any other fixed length you want

# # Pad or truncate sequences to ensure they have the same length
# X_train = pad_sequences(X_train, maxlen=max_seq_length, padding='post', truncating='post', value=0)
# X_val = pad_sequences(X_val, maxlen=max_seq_length, padding='post', truncating='post', value=0)
# X_test = pad_sequences(X_test, maxlen=max_seq_length, padding='post', truncating='post', value=0)


# Pad or truncate sequences to ensure they have the same length
# X_train_padded = [seq[:fixed_seq_length] + [[0] * num_landmarks] * (fixed_seq_length - len(seq)) if len(seq) > fixed_seq_length else seq[:fixed_seq_length] for seq in X_train]
# X_val_padded = [seq[:fixed_seq_length] + [[0] * num_landmarks] * (fixed_seq_length - len(seq)) if len(seq) > fixed_seq_length else seq[:fixed_seq_length] for seq in X_val]
# X_test_padded = [seq[:fixed_seq_length] + [[0] * num_landmarks] * (fixed_seq_length - len(seq)) if len(seq) > fixed_seq_length else seq[:fixed_seq_length] for seq in X_test]

# # Convert lists to numpy arrays
# X_train_padded = np.array(X_train_padded)
# X_val_padded = np.array(X_val_padded)
# X_test_padded = np.array(X_test_padded)

In [36]:
# Convert to numpy arrays
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [37]:
exercise_classes = ['barbell biceps curl','bench press','chest fly machine','deadlift','decline bench press',
                  'hammer curl','hip thrust','incline bench press','lat pulldown','lateral raise',
                  'leg extension','leg raises','plank','pull Up','push-up','romanian deadlift',
                  'russian twist','shoulder press','squat','t bar row','tricep Pushdown','tricep dips']

In [38]:
y_train.shape

(521,)

In [22]:
from keras.models import Sequential
from keras.layers import Conv1D, LSTM, Dense, Dropout, MaxPooling1D, Flatten

num_classes = len(exercise_classes)

# Define the model
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(fixed_seq_length, num_landmarks)),
    MaxPooling1D(2),
    LSTM(64, return_sequences=True),
    LSTM(64),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(exercise_classes), activation='softmax')  # Replace num_classes with the number of exercise classes
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


Epoch 1/20
24/24 [==============================] - 17s 518ms/step - loss: 3.0873 - accuracy: 0.0601 - val_loss: 3.0687 - val_accuracy: 0.0638
Epoch 2/20
24/24 [==============================] - 8s 314ms/step - loss: 3.0711 - accuracy: 0.0628 - val_loss: 3.0406 - val_accuracy: 0.0319
Epoch 3/20
24/24 [==============================] - 4s 172ms/step - loss: 3.0378 - accuracy: 0.0708 - val_loss: 2.9202 - val_accuracy: 0.1170
Epoch 4/20
24/24 [==============================] - 4s 175ms/step - loss: 2.9575 - accuracy: 0.1055 - val_loss: 2.8427 - val_accuracy: 0.1702
Epoch 5/20
24/24 [==============================] - 5s 193ms/step - loss: 2.8985 - accuracy: 0.1295 - val_loss: 2.8175 - val_accuracy: 0.1277
Epoch 6/20
24/24 [==============================] - 4s 161ms/step - loss: 2.8971 - accuracy: 0.1429 - val_loss: 2.8875 - val_accuracy: 0.1702
Epoch 7/20
24/24 [==============================] - 4s 147ms/step - loss: 2.8932 - accuracy: 0.1375 - val_loss: 2.8443 - val_accuracy: 0.1915
Epoch

In [23]:
print("Defining the model")
# Define the model
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(fixed_seq_length, num_landmarks)),
    LSTM(64),
    Dense(len(exercise_classes), activation='softmax')
])

print("compiling the model")
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Training the model")
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

print("evaluating the model")
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


Defining the model
compiling the model
Training the model
Epoch 1/20
24/24 [==============================] - 7s 165ms/step - loss: 3.0876 - accuracy: 0.0561 - val_loss: 3.0797 - val_accuracy: 0.0319
Epoch 2/20
24/24 [==============================] - 3s 135ms/step - loss: 3.0660 - accuracy: 0.0748 - val_loss: 3.0631 - val_accuracy: 0.0319
Epoch 3/20
24/24 [==============================] - 3s 138ms/step - loss: 3.0601 - accuracy: 0.0668 - val_loss: 3.0555 - val_accuracy: 0.0638
Epoch 4/20
24/24 [==============================] - 3s 136ms/step - loss: 2.9940 - accuracy: 0.0761 - val_loss: 2.9865 - val_accuracy: 0.0957
Epoch 5/20
24/24 [==============================] - 3s 138ms/step - loss: 2.9447 - accuracy: 0.0801 - val_loss: 2.8819 - val_accuracy: 0.1915
Epoch 6/20
24/24 [==============================] - 3s 145ms/step - loss: 2.9076 - accuracy: 0.1001 - val_loss: 2.8558 - val_accuracy: 0.1702
Epoch 7/20
24/24 [==============================] - 3s 137ms/step - loss: 2.8887 - accurac

In [39]:
print("Defining the model")
# Define the model
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(fixed_seq_length, num_landmarks)),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(128)),
    Dropout(0.5),
    Dense(len(exercise_classes), activation='softmax')
])

print("compiling the model")
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Training the model")
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

print("evaluating the model")
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

Defining the model
compiling the model
Training the model
Epoch 1/20
17/17 [==============================] - 33s 1s/step - loss: 3.0444 - accuracy: 0.0806 - val_loss: 2.9055 - val_accuracy: 0.1385
Epoch 2/20
17/17 [==============================] - 21s 1s/step - loss: 2.8870 - accuracy: 0.1190 - val_loss: 2.7519 - val_accuracy: 0.0769
Epoch 3/20
17/17 [==============================] - 20s 1s/step - loss: 2.7647 - accuracy: 0.1612 - val_loss: 2.6965 - val_accuracy: 0.0923
Epoch 4/20
17/17 [==============================] - 19s 1s/step - loss: 2.6488 - accuracy: 0.1958 - val_loss: 3.0888 - val_accuracy: 0.0462
Epoch 5/20
17/17 [==============================] - 20s 1s/step - loss: 2.5521 - accuracy: 0.2015 - val_loss: 2.6035 - val_accuracy: 0.1692
Epoch 6/20
17/17 [==============================] - 20s 1s/step - loss: 2.6813 - accuracy: 0.1900 - val_loss: 2.5929 - val_accuracy: 0.2308
Epoch 7/20
17/17 [==============================] - 20s 1s/step - loss: 2.5818 - accuracy: 0.2188 - va

In [27]:
# Define the model
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(fixed_seq_length, num_landmarks)),
    LSTM(128, return_sequences=True),
    LSTM(128, return_sequences=True),
    GlobalAveragePooling1D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(exercise_classes), activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val))

print("evaluating the model")
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

Epoch 1/20
12/12 [==============================] - 20s 1s/step - loss: 3.0717 - accuracy: 0.0748 - val_loss: 3.0147 - val_accuracy: 0.0851
Epoch 2/20
12/12 [==============================] - 13s 1s/step - loss: 2.9862 - accuracy: 0.1202 - val_loss: 2.9233 - val_accuracy: 0.0957
Epoch 3/20
12/12 [==============================] - 14s 1s/step - loss: 2.9321 - accuracy: 0.1402 - val_loss: 2.9038 - val_accuracy: 0.1064
Epoch 4/20
12/12 [==============================] - 14s 1s/step - loss: 2.8701 - accuracy: 0.1429 - val_loss: 2.8325 - val_accuracy: 0.1915
Epoch 5/20
12/12 [==============================] - 11s 950ms/step - loss: 2.7943 - accuracy: 0.1776 - val_loss: 2.7302 - val_accuracy: 0.1915
Epoch 6/20
12/12 [==============================] - 12s 989ms/step - loss: 2.6723 - accuracy: 0.1989 - val_loss: 2.5628 - val_accuracy: 0.2340
Epoch 7/20
12/12 [==============================] - 11s 918ms/step - loss: 2.5770 - accuracy: 0.2310 - val_loss: 2.5018 - val_accuracy: 0.2234
Epoch 8/20


In [ ]:
# from keras.optimizers import RMSprop, Adam, SGD

# for l in [0.001, 0.01, 0.1]:
#     for b in [16, 32, 64]:
#         for d in [0.3, 0.5, 0.7]:
#             for e in [5, 10, 15]:
#                 for op in [Adam(l), RMSprop(l), SGD(l)]:
#                     m = Sequential([
#                         Conv1D(64, 3, activation='relu', input_shape=(max_seq_length, num_landmarks)),
#                         LSTM(128, return_sequences=True),
#                         LSTM(128, return_sequences=True),
#                         GlobalAveragePooling1D(),
#                         Dense(256, activation='relu'),
#                         Dropout(d),
#                         Dense(len(exercise_classes), activation='softmax')
#                     ])
#                     print(l," - ", b," - ", d," - ", e," - ", op)
#                     m.compile(loss='sparse_categorical_crossentropy', optimizer=op, metrics=['accuracy'])
#                     m.fit(X_train, y_train, epochs=e, batch_size=b, validation_data=(X_val, y_val))

In [ ]:
# Save the model
output_dir = '/kaggle/working/'
with open(output_dir + 'lstm_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# Load the model
with open(output_dir + 'lstm_model.pkl', 'rb') as f:
    model = pickle.load(f)